<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M3/exercises/Claim_Similarity_and_Classification_SBERT_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [ ]:
import scipy.spatial
import numpy as np
import os, json
import glob
import re
import torch
import pandas as pd

In [ ]:
df_row = pd.read_csv('claim5000.csv')

FileNotFoundError: ignored

In [ ]:
df_row.head()

In [ ]:
# df_row.iloc[0]['text']

In [ ]:
df_newmergetext = df_row.groupby('patent_id')['text'].apply(lambda x: "{%s}" % ', '.join(x))
# df_newmergetext.head()
# type(df_newmergetext)

In [ ]:
df_newmergetext = df_newmergetext.to_frame()
# df_newmergetext.head()
# df_newmergetext.iloc[0]['text']

In [ ]:
df = pd.merge(df_row.drop_duplicates(subset='patent_id'), df_newmergetext, on='patent_id', how='inner')

In [ ]:
# df.head()
df['text_y'] = df['text_y'].str.strip('{}')

In [ ]:
df = df.drop('text_x', axis=1)

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={"text_y": "text"}, errors="raise")

In [ ]:
total_patents = df.count()
total_patents

In [ ]:
# df_test_tensor = df.drop(['text', 'dependent'], axis=1)
# df_test_tensor
# sorted, indices = torch.sort(df_tensor, 0)

In [ ]:
claims = list(df.text)
patent_id = list(df.patent_id)

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
Test_claim_embeddings = embedder.encode(claims)

In [ ]:
def get_top_n_similar_patents_df(new_claim, claims):
    # embedder = SentenceTransformer('bert-base-nli-stsb-mean-tokens')
    embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')
    query_embeddings = embedder.encode([new_claim])

    # list of patent claims
    claim_embeddings = embedder.encode(claims)

    # get top 100 patent claims based on cosine similarity
    closest_n = 100
    distances = scipy.spatial.distance.cdist(query_embeddings, claim_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    # save similar patents info
    top_claim_ids = []
    top_claims = []
    top_similarity_scores = []


    print('New_claim: ' + new_claim + '\n')

    # Find the closest 100 patent claims for each query new_claim based on cosine similarity
    for idx, distance in results[0:closest_n]:
        top_claim_ids.append(patent_id[idx])
        top_claims.append(claims[idx])
        top_similarity_scores.append(round((1-distance), 4))
        print('Patent ID: ' + str(patent_id[idx]))
        print('PubMed Claim: ' + claims[idx])
        print('Similarity Score: ' + "%.4f" % (1-distance))
        print('\n')
        
    top_100_similar_patents_df = pd.DataFrame({
        'top_claim_ids': top_claim_ids,
        'cosine_similarity': top_similarity_scores,
        'claims': top_claims,
    })
    
    return top_100_similar_patents_df

In [ ]:
new_claim = ['The computer-implemented method of claim 4 wherein the filtering criteria comprises a number of page groups in the page group list is greater than 1, and the representative rank is less than or equal to the exit-entry max paths. The computer-implemented method of claim 2 wherein the selecting of the top n unconverted session paths further comprises only selecting unconverted sessions that meet a filtering criteria. The computer-implemented method of claim 1 further comprising creating a master table that lists the set of load test scenarios and the think times that represents a distribution of paths taken by real users during the peak hour.']

In [ ]:
# search for similar patent claim

if os.path.exists('top_100_similar_patents_df.csv'):
    get_top_n_similar_patents_df = pd.read_csv('top_100_similar_patents_df.csv')

else:
    get_top_n_similar_patents_df = get_top_n_similar_patents_df(new_claim[0], claims)

In [ ]:
# type(get_top_n_similar_patents_df)

In [ ]:
get_top_n_similar_patents_df.head()

In [ ]:
result = pd.concat([get_top_n_similar_patents_df, df], axis=1, join='inner')

In [ ]:
result.head()

In [ ]:
print(result.count())

In [ ]:
# import pandas as pd
# import torch

# # determine the supported device
# def get_device():
#     if torch.cuda.is_available():
#         device = torch.device('cuda:0')
#     else:
#         device = torch.device('cpu') # don't have GPU 
#     return device

# # convert a df to tensor to be used in pytorch
# def df_to_tensor(df):
#     device = get_device()
#     return torch.from_numpy(df.values).float().to(device)

# df_tensor = df_to_tensor(result[['cosine_similarity','claim_number','patent_id']])
# # series_tensor = df_to_tensor(series)

In [ ]:
# df_tensor
# sorted, indices = torch.sort(df_tensor, 0)
# a = torch.topk(df_tensor, 2, dim = 0)
# a

In [ ]:
k_similar_patents = result.nlargest(10, ['cosine_similarity'])

In [ ]:
k_similar_patents

In [ ]:
final_result = k_similar_patents.mode()
print('Prediction for Subclass of New Patent is: ', final_result.iloc[0]['subclass'])